# Process Tail Simulation Results

In [56]:
import os, pickle

results_dir = r"D:\Ergodicity Simulations\2025-10-08 Run - Varying Loss Tail and Limits - $10M"

sample_file = "Ded (100K) - LR (0.6) - Pol_Lim (50M) - X_Th_%le (0.0005) - X_Shape (2.0) - X_Scale (1.0) - 250K Sims - 25 Yrs.pkl"

file_path = os.path.join(results_dir, sample_file)
with open(file_path, "rb") as f:
    loaded_data = pickle.load(f)

In [57]:
loaded_data

SimulationResults(final_assets=array([1.4637041e+07, 1.4895468e+07, 1.5072962e+07, ..., 1.4909375e+07,
       1.4855511e+07, 1.4782316e+07], shape=(250000,), dtype=float32), annual_losses=array([[1.18460430e+05, 1.04350922e+05, 2.84990250e+05, ...,
        3.40381344e+05, 1.97912312e+05, 2.19086688e+05],
       [1.74787402e+04, 1.12905938e+05, 1.68725516e+05, ...,
        2.75829750e+05, 2.59753469e+05, 6.69081406e+04],
       [5.36536641e+04, 1.02061227e+05, 2.01835781e+04, ...,
        3.30333242e+04, 0.00000000e+00, 7.57646875e+04],
       ...,
       [1.50324156e+05, 3.06423000e+05, 3.31056312e+05, ...,
        9.73757812e+04, 7.74627188e+04, 1.25360734e+05],
       [2.40515750e+05, 4.03165062e+05, 1.19188760e+04, ...,
        3.49842500e+04, 9.89353750e+04, 1.29099961e+05],
       [1.87997078e+05, 0.00000000e+00, 5.69903300e+06, ...,
        1.92922425e+06, 1.30938656e+05, 1.11919852e+05]],
      shape=(250000, 25), dtype=float32), insurance_recoveries=array([[0.0000000e+00, 0.000

In [58]:
loaded_data.ruin_probability

{'5': 0.0, '10': 0.0, '15': 0.0, '20': 0.0, '25': 0.0}

In [59]:
loaded_data.growth_rates.mean()

np.float64(0.01535565753865856)

In [60]:
import pickle
import re

from pathlib import Path
from PIL import Image

from ergodic_insurance.monte_carlo import SimulationResults
import numpy as np
from time import perf_counter
from tqdm.auto import tqdm

def _parse_number(text):
    text = text.strip().replace(",", "")
    m = re.fullmatch(r'([+-]?\d+(?:\.\d+)?)([KMB])?$', text, re.I)
    if not m:
        # fallback: plain int/float or leave as-is
        try:
            return int(text)
        except ValueError:
            try:
                return float(text)
            except ValueError:
                return text
    num = float(m.group(1))
    mult = {"K": 1_000, "M": 1_000_000, "B": 1_000_000_000}.get((m.group(2) or "").upper(), 1)
    val = num * mult
    return int(val) if val.is_integer() else val


def parse_config_key(key: str) -> dict:
    parts = re.split(r"\s*-\s*", key.strip())
    out = {}
    for part in parts:
        if not part:
            continue

        # e.g. "Cap (100M)"
        m = re.match(r"^([A-Za-z_%]+)\s*\(\s*([^)]+)\s*\)$", part)
        if m:
            out[m.group(1)] = _parse_number(m.group(2))
            continue

        # e.g. "0K Sims" or "50 Yrs"
        m = re.match(r"^([+-]?\d+(?:\.\d+)?)\s*([KMB])?\s*([A-Za-z_]+)$", part)
        if m:
            value = _parse_number((m.group(1) or "") + (m.group(2) or ""))
            out[m.group(3)] = value
            continue

        # flags like "NOINS"
        if part.upper() == "NOINS":
            out["NOINS"] = True

    return out

results_dir = Path(results_dir)

sample_files = "*.pkl"

pkl_paths = sorted(results_dir.glob(sample_files))

qs = np.arange(0.01, 1.00, 0.01) # Growth Rate Quantiles

all_configurations = {}
if not pkl_paths:
    print(f"No pickle files found in {results_dir}.")
else:
    try:
        iterator = tqdm(pkl_paths, desc="Processing pickle files", unit="file")
    except Exception:
        iterator = pkl_paths  # fallback without progress bar

    start_time = perf_counter()
    for idx, path in enumerate(iterator, 1):
        if idx > 1:
            elapsed = perf_counter() - start_time
            avg = elapsed / (idx - 1)
            remaining = avg * (len(pkl_paths) - (idx - 1))
            if hasattr(iterator, "set_postfix"):
                iterator.set_postfix(avg_s=f"{avg:.2f}", eta_s=f"{remaining:.1f}")
        try:
            with open(path, "rb") as f:
                one_config = pickle.load(f)
                growth_rate = one_config.growth_rates.mean()
                growth_rate_ci = {str(q): val for q, val in zip(qs, np.quantile(one_config.growth_rates, qs))}
                ror = one_config.ruin_probability
                all_configurations[path.stem] = {
                    "growth_rate": growth_rate,
                    "growth_rate_ci": growth_rate_ci,
                    "risk_of_ruin": ror
                }
        except Exception as e:
            print(f"Skipping {path.name}: {e}")
    print(f"Loaded {len(all_configurations)} pickle files into all_configurations.")

parsed_params_by_key = {k: parse_config_key(k) | all_configurations[k] for k in all_configurations.keys()}
parsed_params_by_key

Processing pickle files:   0%|          | 0/120 [00:00<?, ?file/s]

Skipping Ded (100K) - LR (0.5) - Pol_Lim (100M) - X_Th_%le (0.0005) - X_Shape (1.0) - X_Scale (2.0) - 250K Sims - 25 Yrs.pkl: [Errno 13] Permission denied: 'D:\\Ergodicity Simulations\\2025-10-08 Run - Varying Loss Tail and Limits - $10M\\Ded (100K) - LR (0.5) - Pol_Lim (100M) - X_Th_%le (0.0005) - X_Shape (1.0) - X_Scale (2.0) - 250K Sims - 25 Yrs.pkl'
Loaded 119 pickle files into all_configurations.


{'Ded (100K) - LR (0.5) - Pol_Lim (100M) - X_Th_%le (0.0001) - X_Shape (1.0) - X_Scale (1.5) - 250K Sims - 25 Yrs': {'Ded': 100000,
  'LR': 0.5,
  'Pol_Lim': 100000000,
  'X_Th_%le': 0.0001,
  'X_Shape': 1,
  'X_Scale': 1.5,
  'Sims': 250000,
  'Yrs': 25,
  'growth_rate': np.float64(0.004764710459195019),
  'growth_rate_ci': {'0.01': np.float64(0.003247964901383966),
   '0.02': np.float64(0.003467479101382196),
   '0.03': np.float64(0.0035979890706948937),
   '0.04': np.float64(0.003693130696192384),
   '0.05': np.float64(0.0037732052616775034),
   '0.060000000000000005': np.float64(0.0038397120218724014),
   '0.06999999999999999': np.float64(0.003895883189979941),
   '0.08': np.float64(0.003945470079779625),
   '0.09': np.float64(0.003990188417956233),
   '0.09999999999999999': np.float64(0.004030846618115902),
   '0.11': np.float64(0.004068341581150889),
   '0.12': np.float64(0.004103409498929978),
   '0.13': np.float64(0.004136339295655489),
   '0.14': np.float64(0.00416712023317813

In [61]:
outfile = r"cache\parsed_params_by_key.pkl"
out_path = Path(outfile)
if out_path.parent and not out_path.parent.exists():
    out_path.parent.mkdir(parents=True, exist_ok=True)

with open(out_path, "wb") as f:
    pickle.dump(parsed_params_by_key, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Wrote {len(parsed_params_by_key)} configurations to {outfile}")

Wrote 119 configurations to cache\parsed_params_by_key.pkl


In [62]:
# Replace the first failing comprehension with this safe version:
all_lims = sorted({sc['Pol_Lim'] for sc in parsed_params_by_key.values() if 'Pol_Lim' in sc})
all_lims

[50000000, 100000000, 200000000, 500000000]

In [63]:
# Replace the first failing comprehension with this safe version:
all_keys = set().union(*(sc.keys() for sc in parsed_params_by_key.values()))
all_keys

{'Ded',
 'LR',
 'NOINS',
 'Pol_Lim',
 'Sims',
 'X_Scale',
 'X_Shape',
 'X_Th_%le',
 'Yrs',
 'growth_rate',
 'growth_rate_ci',
 'risk_of_ruin'}

In [64]:
for key in all_keys:
    all_key_vals = sorted({str(sc[key]) for sc in parsed_params_by_key.values() if key in sc and type(sc[key]) not in (list, dict, set)})
    if all_key_vals != []:
        print(f"{key}: {all_key_vals}")
    all_key_vals = {type(sc[key]) for sc in parsed_params_by_key.values() if key in sc and type(sc[key]) in (list, dict, set)}
    if all_key_vals != set():
        print(f"{key}: {all_key_vals}")

NOINS: ['True']
risk_of_ruin: {<class 'dict'>}
X_Scale: ['0.5', '1', '1.5', '2']
Yrs: ['25']
growth_rate_ci: {<class 'dict'>}
Pol_Lim: ['100000000', '200000000', '50000000', '500000000']
Sims: ['250000']
growth_rate: ['-0.0007936883942894283', '-0.0008980569296037306', '-0.0023398476077273146', '-0.007044087690290064', '-0.022272200105659664', '-0.03732436784382164', '-0.04773432531663775', '0.0016966180183858608', '0.0017161233670128831', '0.0017373175699001116', '0.0017466425082793792', '0.0017533623901797688', '0.0017543078342181543', '0.0017643801527226287', '0.0017781560429321953', '0.0017809619493305852', '0.0017828466505331298', '0.001792855115322923', '0.0017983402927458632', '0.0018141830017886248', '0.0018151092141088724', '0.001820360321620438', '0.0018231831786494344', '0.001825043844207461', '0.0018297643433796422', '0.0018298718346296489', '0.0018397771739101835', '0.001860356538029371', '0.0018639623894387797', '0.001871031610160232', '0.0018747518621448653', '0.00188625

I have the following set of data as a list of dictionaries for 1029 configurations that I'd like to explore visually in Python:

```
'risk_of_ruin': {<class 'dict'>} # Risk of Ruin snapshots at 5-year intervales (5, 10, 15, 20, 25)
'Ded': ['100000'] # Deductible
'X_Scale': ['0.5', '1', '1.5', '2'] # Scale parameter for the Generalized Pareto Distribution (GPD) tail
'X_Th_%le': ['0.0001', '0.0005', '0.001', 'None'] # Threshold percentile for the Generalized Pareto Distribution (GPD) tail
'growth_rate': <class 'float'> # Mean Growth Rate at the end of the simulation
'growth_rate_ci': {<class 'dict'>} # Growth Rate quantiles from 0.01 to 0.99 in increments of 0.01 at the end of the simulation
'Pol_Lim': ['100000000', '200000000', '50000000', '500000000']
'X_Shape': ['0', '1', '1.5', '2', '2.5'] # Shape parameter for the Generalized Pareto Distribution (GPD) tail
'LR': ['0.3', '0.4', '0.5', '0.6', '0.7'] # Loss Ratio (Claims / Premiums), lower implies a higher premium charge for the same expected losses
'Yrs': ['25'] # Simulations were run for 25 years only
'Sims': ['100000'] # Each configuration was run for 100,000 simulations
# There are also configurations without insurance for which there is no 'Pol_Lim', no 'Ded', and no 'LR' parameters. These scenarios are marked with `'NOINS': True`
```

For the first set of plots, I'd like to create an Efficiency Frontier plot with the following parameters:
x-axix: 'risk_of_ruin'
y-axis: 'growth_rate'
Display this curve for each tail configuration in the following setup:
row: 'X_Th_%le'
column: 'X_Shape'
Plot different graphs for each collection of 'X_Scale', which are transformations on the loss distribution with '1' representing a close match, '0.5' representing 0.5 loss density compared to default tail, '2' representing twice the density of the default tail, and so on.
Color represents percentiles, using the pallette "cividis"
Encode different limits as shapes for the median/mean/quantile, so for example, a triangle in different colors represents the same limit (50M)
Shapes are as follows:
'NOINS': plus sign, "P" shape code
'50000000': triangle, "v" shape code
'100000000': square, "s" shape code
'200000000': pentagon, "p" shape code
'500000000': hexagon, "H" shape code

There should be 4 plots (one for each 'X_Scale') with DPI set to 300px that get saved to individual PNG files.


In [65]:
ins_only_vals = [val for val in parsed_params_by_key.values() if val.get('NOINS', False) is False]
len(ins_only_vals)

75

In [66]:
[c for c in ins_only_vals if c['X_Shape'] == 0]


[]

In [67]:
x_shapes = sorted(set(c['X_Shape'] for c in ins_only_vals))
x_shapes

[1, 1.5, 2, 2.5]

In [68]:
[c for c in ins_only_vals if c['X_Shape'] == 0]

[]

In [69]:
x_thresholds = sorted({(c['X_Th_%le'] if isinstance(c['X_Th_%le'], (float, np.floating)) else 0) for c in ins_only_vals})
x_thresholds

[0.0001, 0.0005, 0.001]

In [70]:
lr = sorted(set(c['LR'] for c in ins_only_vals))
lr

[0.5, 0.6, 0.7]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from matplotlib.lines import Line2D

# Assuming your data is stored in a list called 'configurations'
# configurations = [{'risk_of_ruin': {...}, 'growth_rate': ..., 'X_Scale': ..., ...}, ...]


def create_efficiency_frontier_plots(configurations):
    """
    Create efficiency frontier plots for actuarial simulation results.

    Parameters:
    -----------
    configurations : list of dict
        List of configuration dictionaries with simulation results

    Output:
    -------
    Saves PNG files for each X_Scale with efficiency frontier plots.
    """

    # Define shape mappings
    shape_map = {
        # 'NOINS': 'P',      # Plus sign
        50000000: 'v',   # Triangle
        100000000: 's',  # Square
        200000000: 'p',  # Pentagon
        500000000: 'H'   # Hexagon
    }

    # Get unique values for separate charts
    x_scales = sorted(set(c['X_Scale'] for c in configurations))
    lr = sorted(set(c['LR'] for c in configurations))

    # Define percentiles for overlay
    highlight_percentiles = [0.50, 0.75, 0.90, 0.95, 0.99]
    legend_percentiles = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]

    # Create a plot for each X_Scale
    for scale in x_scales:
        for loss_ratio in lr:
            config_subset = [c for c in configurations if c['X_Scale'] == scale and c['LR'] == loss_ratio]
            # Get unique values for faceting
            x_shapes = sorted(set(float(c['X_Shape']) for c in config_subset))
            x_shapes = [s for s in x_shapes if s != 0] # Filter out 0
            x_thresholds = sorted({(c['X_Th_%le'] if isinstance(c['X_Th_%le'], (float, np.floating)) else 0.0) for c in config_subset})
            x_thresholds = [s for s in x_thresholds if s != 0] # Filter out 0

            # all_gr_min, all_gr_max = [], []
            # for val in ins_only_vals:
            #     val_min, val_max = min(val['growth_rate_ci'].values()), max(val['growth_rate_ci'].values())
            #     all_gr_min.append(val_min)
            #     all_gr_max.append(val_max)

            # x_min, x_max = min(all_gr_min), max(all_gr_max)

            all_ror_min, all_ror_max = [], []
            for val in config_subset:
                val_min, val_max = min(val['risk_of_ruin'].values()), max(
                    val['risk_of_ruin'].values())
                all_ror_min.append(val_min)
                all_ror_max.append(val_max)

            y_min, y_max = min(all_ror_min), max(all_ror_max)

            # Filter configurations for this scale
            scale_configs = config_subset

            # Create figure with subplots
            n_rows = len(x_thresholds)
            n_cols = len(x_shapes)

            fig, axes = plt.subplots(n_rows, n_cols,
                                    figsize=(5*n_cols, 4*n_rows),
                                    dpi=300,
                                    squeeze=False)

            # Set overall title
            scale_label = f"{float(scale):.1f}x" if scale != '1' else "Baseline"
            lr_pct = f"{loss_ratio*100:.0f}%"
            fig.suptitle(f'Efficiency Frontier - Scale: {scale_label} - Loss Ratio: {lr_pct}',
                        fontsize=16, fontweight='bold', y=0.995)

            # Create colormap for percentiles
            cmap = plt.cm.viridis

            # Plot each facet
            for i, threshold in enumerate(x_thresholds):
                for j, shape_param in enumerate(x_shapes):
                    ax = axes[i, j]

                    # Debug: Check what values we're comparing
                    if threshold == 0 or shape_param == 0:
                        print(
                            f"\n=== Debug for threshold={threshold}, shape_param={shape_param} ===")
                        print(
                            f"shape_param type: {type(shape_param)}, value: {repr(shape_param)}")
                        print(
                            f"threshold type: {type(threshold)}, value: {repr(threshold)}")

                        # Sample a few configs to see their actual values
                        for c in scale_configs[:3]:
                            print(
                                f"  X_Shape: {repr(c['X_Shape'])} (type: {type(c['X_Shape'])})")
                            print(
                                f"  X_Th_%le: {repr(c['X_Th_%le'])} (type: {type(c['X_Th_%le'])})")
                            print(
                                f"  float(c['X_Shape']) == shape_param: {float(c['X_Shape']) == shape_param}")
                            print(
                                f"  Threshold match: {c['X_Th_%le'] == threshold or (c['X_Th_%le'] is None and threshold == 0)}")
                            print()

                    # Filter for this facet
                    facet_configs = [c for c in scale_configs
                                    if (c['X_Th_%le'] == threshold or (c['X_Th_%le'] == 'None' and threshold == 0))
                                    and float(c['X_Shape']) == shape_param]

                    if (threshold == 0 or shape_param == 0):
                        print(len(facet_configs))

                    # First pass: Plot all percentiles as heatmap (background)
                    for config in facet_configs:
                        # Get risk of ruin at year 25
                        # ror_25 = config['risk_of_ruin'].get(25, config['risk_of_ruin'].get('25'))

                        # Calculate log of policy limit for x-axis
                        if config.get('NOINS', False):
                            x_value = 0  # or np.nan if you want to exclude NOINS from plot
                        else:
                            pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                                config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                            x_value = np.log10(float(pol_lim))

                        # Plot all percentiles from growth_rate_ci as small dots
                        if 'growth_rate_ci' in config and config['growth_rate_ci']:
                            for pct_key, growth_value in config['growth_rate_ci'].items():
                                # Convert key to float percentile
                                try:
                                    pct = float(pct_key)
                                    color = cmap(pct)

                                    # Plot as small dot for heatmap effect
                                    ax.scatter(x_value, growth_value,
                                                c=[color],
                                                s=15,
                                                alpha=0.6,
                                                edgecolors='none')
                                except (ValueError, TypeError):
                                    continue

                    # Second pass: Overlay specific percentiles with shapes
                    for config in facet_configs:
                        # Get risk of ruin at year 25
                        # ror_25 = config['risk_of_ruin'].get(25, config['risk_of_ruin'].get('25'))

                        # Calculate log of policy limit for x-axis
                        if config.get('NOINS', False):
                            x_value = 0  # or np.nan if you want to exclude NOINS from plot
                        else:
                            pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                                config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                            x_value = np.log10(float(pol_lim))

                        # Determine shape based on limit
                        if config.get('NOINS', False):
                            marker = shape_map['NOINS']
                            marker_size = 150
                        else:
                            pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                                config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                            marker = shape_map.get(pol_lim, 'x')
                            marker_size = 120

                        # Plot specific percentiles with shapes
                        if 'growth_rate_ci' in config and config['growth_rate_ci']:
                            for pct in highlight_percentiles:
                                pct_key = f"{pct:.2f}" if f"{pct:.2f}" in config['growth_rate_ci'] else pct
                                if pct_key in config['growth_rate_ci']:
                                    growth = config['growth_rate_ci'][pct_key]
                                    color = cmap(pct)

                                    ax.scatter(x_value, growth,
                                                marker=marker,
                                                c=[color],
                                                s=marker_size,
                                                alpha=0.6,
                                                edgecolors='black',
                                                linewidths=1.5)

                        # Plot mean in red
                        if 'growth_rate' in config:
                            growth_mean = config['growth_rate']
                            ax.scatter(x_value, growth_mean,
                                        marker=marker,
                                        c='red',
                                        s=marker_size,
                                        alpha=0.6,
                                        edgecolors='black',
                                        linewidths=1.5,
                                        zorder=10)

                    # Formatting
                    ax.set_xlabel('Policy Limit (log scale)', fontsize=10)
                    ax.set_ylabel('Growth Rate', fontsize=10)
                    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

                    # Set title for each subplot
                    thresh_label = threshold if threshold != 'None' else 'No Tail'
                    ax.set_title(f'Threshold: {thresh_label}\nShape: {shape_param}',
                                fontsize=9)

                    # Set axis limits
                    # ax.set_xlim(x_min, x_max)
                    ax.set_xlim(np.log10(50000000* 0.9),
                                np.log10(500000000 * 1.1))
                    ax.set_ylim(y_min, y_max)

                    # Set x-ticks to show actual policy limit values
                    ax.set_xticks([np.log10(50e6), np.log10(
                        100e6), np.log10(200e6), np.log10(500e6)])
                    ax.set_xticklabels(['$50M', '$100M', '$200M', '$500M'])

            # Create legends
            # Shape legend
            shape_elements = []
            for limit_name, limit_value in [  # ('No Insurance', 'NOINS'),
                    ('$50M', 50000000),
                    ('$100M', 100000000),
                    ('$200M', 200000000),
                    ('$500M', 500000000)]:
                shape_elements.append(Line2D([0], [0], marker=shape_map[limit_value],
                                            color='w', markerfacecolor='gray',
                                            markersize=10, label=limit_name,
                                            markeredgecolor='black', linewidth=1))

            # Color legend (percentiles + mean)
            color_elements = []
            for pct in legend_percentiles:
                color_elements.append(mpatches.Patch(color=cmap(pct),
                                                    label=f'{int(pct*100)}th %ile',
                                                    edgecolor='black',
                                                    linewidth=0.5))
            color_elements.append(mpatches.Patch(color='red',
                                                label='Mean',
                                                edgecolor='black',
                                                linewidth=0.5))

            # Add legends to the figure
            fig.legend(handles=shape_elements, loc='upper left',
                        bbox_to_anchor=(0.0, 0.0), title='Policy Limit',
                        frameon=False, fontsize=9, ncol=len(shape_elements))
            fig.legend(handles=color_elements, loc='upper right',
                        bbox_to_anchor=(1.0, 0.0), title='Percentile',
                        frameon=False, fontsize=9, ncol=len(color_elements))

            plt.tight_layout(rect=[0, 0, 1, 0.99])

            # Save figure
            filename = f'cache/efficiency_frontier_scale_{str(scale).replace(".", "p")}_lr_{str(loss_ratio).replace(".", "p")}.png'
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            print(f'Saved: {filename}')
            plt.close()


create_efficiency_frontier_plots(ins_only_vals)

C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformatio

Saved: cache/efficiency_frontier_scale_0p5_lr_0p3.png


C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformatio

Saved: cache/efficiency_frontier_scale_0p5_lr_0p4.png


C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformatio

...

Saved: cache/efficiency_frontier_scale_2_lr_0p5.png


C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformatio

Saved: cache/efficiency_frontier_scale_2_lr_0p6.png


C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(y_min, y_max)
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\4062665780.py:218: UserWarning: Attempting to set identical low and high ylims makes transformatio

Saved: cache/efficiency_frontier_scale_2_lr_0p7.png


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

from matplotlib.lines import Line2D
from scipy.interpolate import interp1d

# Assuming your data is sto#cc0000 in a list called 'configurations'
# configurations = [{'risk_of_ruin': {...}, 'growth_rate': ..., 'X_Scale': ..., ...}, ...]


def create_efficiency_frontier_plots_base_only(configurations):
    """
    Create efficiency frontier plots for actuarial simulation results.

    Parameters:
    -----------
    configurations : list of dict
        List of configuration dictionaries with simulation results

    Output:
    -------
    Saves PNG files for each X_Scale with efficiency frontier plots.
    """

    # Get unique values for separate charts
    x_scales = sorted(set(c['X_Scale'] for c in configurations))
    lr = sorted(set(c['LR'] for c in configurations))

    # Define percentiles for overlay
    highlight_percentiles = [0.50, 0.75, 0.90, 0.95, 0.99]
    legend_percentiles = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
    all_percentiles = np.arange(0.01, 1.00, 0.01).tolist()

    # Create a plot for each X_Scale
    for scale in x_scales:
        for loss_ratio in lr:
            if scale != 2.0 or loss_ratio != 0.7:
                continue

            config_subset = [c for c in configurations if c['X_Scale'] == scale and c['LR'] == loss_ratio]
            # Get unique values for faceting
            x_shapes = sorted(set(float(c['X_Shape']) for c in config_subset))
            x_shapes = [s for s in x_shapes if s != 0] # Filter out 0
            base_shape = 2.5
            x_shapes = [base_shape]
            x_thresholds = sorted({(c['X_Th_%le'] if isinstance(c['X_Th_%le'], (float, np.floating)) else 0.0) for c in config_subset})
            x_thresholds = [s for s in x_thresholds if s != 0] # Filter out 0
            base_threshold = 0.0001
            x_thresholds = [base_threshold]

            # all_gr_min, all_gr_max = [], []
            # for val in ins_only_vals:
            #     val_min, val_max = min(val['growth_rate_ci'].values()), max(val['growth_rate_ci'].values())
            #     all_gr_min.append(val_min)
            #     all_gr_max.append(val_max)

            # x_min, x_max = min(all_gr_min), max(all_gr_max)

            # Filter configurations for this scale
            scale_configs = config_subset

            # Create figure with subplots
            n_rows = len(x_thresholds)
            n_cols = len(x_shapes)

            fig, axes = plt.subplots(n_rows, n_cols,
                                    figsize=(5*3, 4*4),
                                    dpi=300,
                                    squeeze=False)

            # Set overall title
            scale_label = f"{float(scale):.1f}x" if scale != '1' else "Baseline"
            lr_pct = f"{loss_ratio*100:.0f}%"
            threshold_pct = f"{base_threshold*100:.2f}%"
            fig.suptitle(f'Efficiency Frontier - Scale: {scale_label} - Loss Ratio: {lr_pct} - Tail Threshold: {threshold_pct} - Shape: {base_shape}',
                        fontsize=16, fontweight='bold', y=0.995)

            # Create colormap for percentiles
            cmap = plt.cm.viridis

            # Plot each facet
            for i, threshold in enumerate(x_thresholds):
                for j, shape_param in enumerate(x_shapes):
                    ax = axes[i, j]

                    # Debug: Check what values we're comparing
                    if threshold == 0 or shape_param == 0:
                        print(
                            f"\n=== Debug for threshold={threshold}, shape_param={shape_param} ===")
                        print(
                            f"shape_param type: {type(shape_param)}, value: {repr(shape_param)}")
                        print(
                            f"threshold type: {type(threshold)}, value: {repr(threshold)}")

                        # Sample a few configs to see their actual values
                        for c in scale_configs[:3]:
                            print(
                                f"  X_Shape: {repr(c['X_Shape'])} (type: {type(c['X_Shape'])})")
                            print(
                                f"  X_Th_%le: {repr(c['X_Th_%le'])} (type: {type(c['X_Th_%le'])})")
                            print(
                                f"  float(c['X_Shape']) == shape_param: {float(c['X_Shape']) == shape_param}")
                            print(
                                f"  Threshold match: {c['X_Th_%le'] == threshold or (c['X_Th_%le'] is None and threshold == 0)}")
                            print()

                    # Filter for this facet
                    facet_configs = [c for c in scale_configs
                                    if (c['X_Th_%le'] == threshold or (c['X_Th_%le'] == 'None' and threshold == 0))
                                    and float(c['X_Shape']) == shape_param]

                    if (threshold == 0 or shape_param == 0):
                        print(len(facet_configs))

                    # # First pass: Plot all percentiles as heatmap (background)
                    # for config in facet_configs:
                    #     # Get risk of ruin at year 25
                    #     # ror_25 = config['risk_of_ruin'].get(25, config['risk_of_ruin'].get('25'))

                    #     # Calculate log of policy limit for x-axis
                    #     if config.get('NOINS', False):
                    #         x_value = 0  # or np.nan if you want to exclude NOINS from plot
                    #     else:
                    #         pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                    #             config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                    #         x_value = np.log10(float(pol_lim))

                    #     # Plot all percentiles from growth_rate_ci as small dots
                    #     if 'growth_rate_ci' in config and config['growth_rate_ci']:
                    #         for pct_key, growth_value in config['growth_rate_ci'].items():
                    #             # Convert key to float percentile
                    #             try:
                    #                 pct = float(pct_key)
                    #                 color = cmap(pct)

                    #                 # Plot as small dot for heatmap effect
                    #                 ax.scatter(x_value, growth_value,
                    #                             c=[color],
                    #                             s=15,
                    #                             alpha=1.0,
                    #                             edgecolors='none')
                    #             except (ValueError, TypeError):
                    #                 continue

                    # # Preliminary Interpolation - Draw lines connecting same percentiles across limits
                    # # Organize data by percentile
                    # percentile_data = {pct: [] for pct in all_percentiles}
                    
                    # for config in facet_configs:
                    #     # Calculate log of policy limit for x-axis
                    #     if config.get('NOINS', False):
                    #         continue  # Skip NOINS for line plotting
                    #     else:
                    #         pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                    #             config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                    #         x_value = np.log10(float(pol_lim))
                        
                    #     # Collect percentile values
                    #     if 'growth_rate_ci' in config and config['growth_rate_ci']:
                    #         for pct in all_percentiles:
                    #             pct_key = f"{pct:.2f}" if f"{pct:.2f}" in config['growth_rate_ci'] else pct
                    #             if pct_key in config['growth_rate_ci']:
                    #                 growth = config['growth_rate_ci'][pct_key]
                    #                 percentile_data[pct].append((x_value, growth, float(pol_lim)))
                    
                    # # Draw lines for each percentile
                    # for pct in all_percentiles:
                    #     if len(percentile_data[pct]) > 1:  # Need at least 2 points for a line
                    #         # Sort by policy limit to ensure proper line ordering
                    #         sorted_points = sorted(percentile_data[pct], key=lambda p: p[2])
                    #         x_coords = [p[0] for p in sorted_points]
                    #         y_coords = [p[1] for p in sorted_points]
                            
                    #         color = cmap(pct)
                    #         ax.plot(x_coords, y_coords, 
                    #                 color=color, 
                    #                 linewidth=2, 
                    #                 alpha=1.0,
                    #                 zorder=4)

                    # # Intermediate pass - Shade regions between consecutive percentiles
                    # # Organize data by percentile
                    # percentile_data = {pct: [] for pct in all_percentiles}

                    # for config in facet_configs:
                    #     # Calculate log of policy limit for x-axis
                    #     if config.get('NOINS', False):
                    #         continue  # Skip NOINS for line plotting
                    #     else:
                    #         pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                    #             config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                    #         x_value = np.log10(float(pol_lim))
                        
                    #     # Collect percentile values
                    #     if 'growth_rate_ci' in config and config['growth_rate_ci']:
                    #         for pct in all_percentiles:
                    #             pct_key = f"{pct:.2f}" if f"{pct:.2f}" in config['growth_rate_ci'] else pct
                    #             if pct_key in config['growth_rate_ci']:
                    #                 growth = config['growth_rate_ci'][pct_key]
                    #                 percentile_data[pct].append((x_value, growth, float(pol_lim)))

                    # # Sort percentiles
                    # sorted_percentiles = sorted(all_percentiles)

                    # # Fill between consecutive percentiles
                    # for idx in range(len(sorted_percentiles) - 1):
                    #     pct_lower = sorted_percentiles[idx]
                    #     pct_upper = sorted_percentiles[idx + 1]
                        
                    #     # Check if we have data for both percentiles
                    #     if len(percentile_data[pct_lower]) > 1 and len(percentile_data[pct_upper]) > 1:
                    #         # Sort by policy limit to ensure proper ordering
                    #         sorted_lower = sorted(percentile_data[pct_lower], key=lambda p: p[2])
                    #         sorted_upper = sorted(percentile_data[pct_upper], key=lambda p: p[2])
                            
                    #         x_coords = [p[0] for p in sorted_lower]
                    #         y_lower = [p[1] for p in sorted_lower]
                    #         y_upper = [p[1] for p in sorted_upper]
                            
                    #         # Use the midpoint color between the two percentiles
                    #         mid_pct = (pct_lower + pct_upper) / 2
                    #         color = cmap(mid_pct)
                            
                    #         ax.fill_between(x_coords, y_lower, y_upper, 
                    #                     color=color, 
                    #                     alpha=1.0,
                    #                     zorder=5,
                    #                     linewidth=0,
                    #                     edgecolor='none')




                    # Intermediate pass - Create smooth gradient using pcolormesh
                    # Organize data by percentile
                    percentile_data = {pct: [] for pct in all_percentiles}

                    for config in facet_configs:
                        # Calculate log of policy limit for x-axis
                        if config.get('NOINS', False):
                            continue  # Skip NOINS
                        else:
                            pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                                config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                            x_value = np.log10(float(pol_lim))
                        
                        # Collect percentile values
                        if 'growth_rate_ci' in config and config['growth_rate_ci']:
                            for pct in all_percentiles:
                                pct_key = f"{pct:.2f}" if f"{pct:.2f}" in config['growth_rate_ci'] else pct
                                if pct_key in config['growth_rate_ci']:
                                    growth = config['growth_rate_ci'][pct_key]
                                    percentile_data[pct].append((x_value, growth, float(pol_lim)))

                    # Get unique x values (policy limits) and sort
                    unique_x_values = sorted(set(p[0] for pct_list in percentile_data.values() for p in pct_list))

                    if len(unique_x_values) > 1:
                        # Create interpolation grid with many points for smooth gradient
                        x_interp = np.linspace(min(unique_x_values), max(unique_x_values), 200)
                        
                        # For each percentile, interpolate growth rates across policy limits
                        from scipy.interpolate import interp1d
                        
                        y_grid = []
                        valid_percentiles = []
                        
                        for pct in sorted(all_percentiles):
                            if len(percentile_data[pct]) > 1:
                                # Sort by x value
                                sorted_data = sorted(percentile_data[pct], key=lambda p: p[0])
                                x_data = np.array([p[0] for p in sorted_data])
                                y_data = np.array([p[1] for p in sorted_data])
                                
                                # Interpolate
                                f = interp1d(x_data, y_data, kind='linear', fill_value='extrapolate')
                                y_interp = f(x_interp)
                                y_grid.append(y_interp)
                                valid_percentiles.append(pct)
                        
                        if len(y_grid) > 1:
                            # Convert to 2D array: rows = percentiles, columns = x positions
                            y_grid = np.array(y_grid)
                            
                            # Create meshgrid for pcolormesh
                            X, Y = np.meshgrid(x_interp, valid_percentiles)
                            
                            # Use pcolormesh with the colormap
                            mesh = ax.pcolormesh(X, y_grid, Y, 
                                                cmap=cmap, 
                                                shading='gouraud',  # Smooth interpolation
                                                vmin=0, vmax=1,
                                                zorder=5)




                    # Second pass: Overlay specific percentiles with shapes
                    for config in facet_configs:
                        # Get risk of ruin at year 25
                        # ror_25 = config['risk_of_ruin'].get(25, config['risk_of_ruin'].get('25'))

                        # Calculate log of policy limit for x-axis
                        if config.get('NOINS', False):
                            x_value = 0  # or np.nan if you want to exclude NOINS from plot
                        else:
                            pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                                config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                            x_value = np.log10(float(pol_lim))

                        # Plot mean in #cc0000
                        if 'growth_rate' in config:
                            growth_mean = config['growth_rate']
                            ax.scatter(x_value, growth_mean,
                                        marker='o',
                                        c='#cc0000',
                                        alpha=1.0,
                                        edgecolors='#cc0000',
                                        linewidths=1.5,
                                        zorder=10)

                    # Formatting
                    ax.set_xlabel('Policy Limit (log scale)', fontsize=10)
                    ax.set_ylabel('Growth Rate', fontsize=10)
                    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

                    # Set title for each subplot
                    # thresh_label = threshold if threshold != 'None' else 'No Tail'
                    # ax.set_title(f'Threshold: {thresh_label}\nShape: {shape_param}',
                    #             fontsize=9)

                    # Set axis limits
                    # ax.set_xlim(x_min, x_max)
                    ax.set_xlim(np.log10(50000000* 0.9),
                                np.log10(500000000 * 1.1))
                    # ax.set_ylim(y_min, y_max)

                    # Set x-ticks to show actual policy limit values
                    ax.set_xticks([np.log10(50e6), np.log10(
                        100e6), np.log10(200e6), np.log10(500e6)])
                    ax.set_xticklabels(['$50M', '$100M', '$200M', '$500M'])

            # Color legend (percentiles + mean)
            color_elements = []
            for pct in legend_percentiles:
                color_elements.append(mpatches.Patch(color=cmap(pct),
                                                    label=f'{int(pct*100)}th %ile',
                                                    edgecolor='black',
                                                    linewidth=0.5))
            color_elements.append(mpatches.Patch(color='#cc0000',
                                                label='Mean',
                                                edgecolor='#cc0000',
                                                linewidth=0.5))

            fig.legend(handles=color_elements, loc='upper center',
                        bbox_to_anchor=(0.5, 0.0), title='Percentile',
                        frameon=False, fontsize=9, ncol=len(color_elements))

            plt.tight_layout(rect=[0, 0, 1, 0.99])

            # Save figure
            filename = f'cache/efficiency_frontier_base.png'
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            print(f'Saved: {filename}')
            plt.close()


create_efficiency_frontier_plots_base_only(ins_only_vals)

C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\2343232132.py:347: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  color_elements.append(mpatches.Patch(color=cmap(pct),
C:\Users\alexf\AppData\Local\Temp\ipykernel_51760\2343232132.py:351: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  color_elements.append(mpatches.Patch(color='#cc0000',


Saved: cache/efficiency_frontier_base.png
